In [64]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
#define data generators for train, validation and test sets
train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/brain_tumor/train',
                                                  target_size=(IMG_SIZE,IMG_SIZE),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='binary',
                                                  subset='training')

val_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/brain_tumor/train',
                                                target_size=(IMG_SIZE,IMG_SIZE),
                                                batch_size=BATCH_SIZE,
                                                class_mode='binary',
                                                subset='validation')

test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/MyDrive/brain_tumor/test',
                                                target_size=(IMG_SIZE,IMG_SIZE),
                                                batch_size=BATCH_SIZE,
                                                class_mode='binary')


Found 2408 images belonging to 2 classes.
Found 602 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [71]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [72]:
#compile model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [77]:
history = model.fit(train_generator,validation_data=val_generator,epochs=1)

76/76 [==============================] - 296s 4s/step - loss: 0.0145 - accuracy: 0.9950 - val_loss: 0.0409 - val_accuracy: 0.9867


In [78]:
model.save("model.h5","label.txt")

In [82]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#load the saved model
model=load_model('/content/model.h5')

#load and preprocess the test image
test_image_path='/content/drive/MyDrive/brain_tumor/test/pred/pred53.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)   #add batch dimension
img_array /=255.  #Normalize the pixel values

 #make predictions
prediction=model.predict(img_array)

 #print the prediction
if prediction <0.5:
  print("prediction:No tumor (probability:",prediction[0][0], ")")
else:
  print("prediction: tumor present (probability:",prediction[0][0], ")")



1/1 [==============================] - 0s 158ms/step
prediction:No tumor (probability: 1.1615836e-05 )
